In [2]:
# create_dummy_models.py
import pickle
from sklearn.linear_model import LogisticRegression
import numpy as np

# Dummy Model 1
model1 = LogisticRegression()
# Create some dummy data to fit the model
X_dummy = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
y_dummy = np.array([0, 1, 0, 1])
model1.fit(X_dummy, y_dummy)

with open('scaler.pkl', 'wb') as f:
    pickle.dump(model1, f)

print("scaler.pkl created.")

# Dummy Model 2 (e.g., a different model or trained on different data)
model2 = LogisticRegression()
X_dummy_2 = np.array([[10, 11], [12, 13], [14, 15], [16, 17]])
y_dummy_2 = np.array([1, 0, 1, 0])
model2.fit(X_dummy_2, y_dummy_2)

with open('churn_model.pkl', 'wb') as f:
    pickle.dump(model2, f)

print("churn_model.pkl created.")

scaler.pkl created.
churn_model.pkl created.


In [7]:
from flask import Flask, request, jsonify, render_template
import pickle
import numpy as np
import sys # For error logging

app = Flask(__name__)

# --- Global variables for loaded models ---
# Initialize to None, they will be loaded on startup
scaler_model = None
churn_prediction_model = None

# --- Function to load models ---
def load_models():
    global scaler_model, churn_prediction_model
    try:
        # Load the StandardScaler
        with open('scaler.pkl', 'rb') as f:
            scaler_model = pickle.load(f)
        print("scaler.pkl loaded successfully.")

        # Load the Churn Prediction Model
        with open('churn_model.pkl', 'rb') as f:
            churn_prediction_model = pickle.load(f)
        print("churn_model.pkl loaded successfully.")

    except FileNotFoundError as e:
        print(f"CRITICAL ERROR: A required model file was not found: {e}. Please ensure 'scaler.pkl' and 'churn_model.pkl' are in the same directory as 'app.py'.", file=sys.stderr)
        sys.exit(1) # Exit if critical models are not found
    except Exception as e:
        print(f"CRITICAL ERROR: An unexpected error occurred while loading models: {e}", file=sys.stderr)
        sys.exit(1) # Exit on any other critical loading error

# Load models when the application starts
# This block runs once when app.py is executed
with app.app_context():
    load_models()

# --- API Endpoints ---

@app.route('/')
def home():
    """
    Renders the main interface page for churn prediction.
    Assumes 'index.html' exists in a 'templates' subfolder.
    """
    return render_template('index.html')

@app.route('/predict_churn', methods=['POST'])
def predict_churn():
    """
    API endpoint to predict churn using the loaded scaler and churn model.
    Expects JSON input with 'features' key containing a list of numbers.
    The number of features must match what the model was trained on.
    """
    # Check if models were loaded successfully at startup
    if scaler_model is None or churn_prediction_model is None:
        return jsonify({'error': 'Models are not loaded. Server startup failed or models are missing.'}), 500

    try:
        data = request.get_json()
        if not data or 'features' not in data:
            return jsonify({'error': 'Invalid input: JSON with "features" key (list of numbers) expected.'}), 400

        input_features = np.array(data['features'])

        # --- IMPORTANT: Adjust 'expected_features_count' to your actual model's input size ---
        # For example, if your model was trained on 10 features, set this to 10.
        expected_features_count = 5 # <--- CHANGE THIS TO THE ACTUAL NUMBER OF FEATURES YOUR MODEL EXPECTS
        if input_features.shape[0] != expected_features_count:
            return jsonify({
                'error': f'Invalid number of features. Expected {expected_features_count} but received {input_features.shape[0]}.',
                'received_features': input_features.tolist()
            }), 400

        # Reshape for single prediction (1 sample, N features)
        reshaped_features = input_features.reshape(1, -1)

        # Step 1: Scale the features using the loaded scaler
        scaled_features = scaler_model.transform(reshaped_features)

        # Step 2: Make prediction with the churn model
        prediction = churn_prediction_model.predict(scaled_features).tolist()
        probability = churn_prediction_model.predict_proba(scaled_features).tolist() if hasattr(churn_prediction_model, 'predict_proba') else None

        # Return results
        return jsonify({
            'input_features': input_features.tolist(),
            'scaled_features_for_model': scaled_features.tolist(), # Can be useful for debugging
            'churn_prediction': prediction[0], # Assuming single prediction, take first element
            'churn_probability': probability[0] if probability else None # Assuming binary classification, first sample
        })

    except ValueError as ve:
        # Catch errors if input_features cannot be converted to numpy array or reshaped
        print(f"ValueError during prediction: {ve}", file=sys.stderr)
        return jsonify({'error': f'Data conversion error: {str(ve)}. Ensure all features are numeric.'}), 400
    except Exception as e:
        # Catch any other unexpected errors during prediction
        print(f"An unexpected error occurred during prediction: {e}", file=sys.stderr)
        return jsonify({'error': f'An internal server error occurred during prediction: {str(e)}. Check server logs for details.'}), 500

if __name__ == '__main__':
    # Run the Flask development server
    # debug=True allows automatic reloading on code changes and provides a debugger
    app.run(debug=True)

scaler.pkl loaded successfully.
churn_model.pkl loaded successfully.
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1